In [1]:
import json
import dspy


with open('gen-ai-ucu-2024-task-3/zno.train.jsonl', 'r') as json_file:
    json_list = list(json_file)

all_questions = []
for json_str in json_list:
    result = json.loads(json_str)
    all_questions.append(result)

all_examples = []
for sample in all_questions:
    example = dspy.Example(
        question=sample['question'],
        options=sample['answers'],
        subject = sample['subject'],
        correct_answer = sample['correct_answers'][0]
    ).with_inputs("question", "options")
    all_examples.append(example)

/home/nazara/Data/UCU/FINETUNE_GENAI/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_set, test_set = all_examples[int(len(all_questions)*0.2):], all_examples[:int(len(all_questions)*0.2)]


In [ ]:
#  vllm serve unsloth/Meta-Llama-3.1-8B-bnb-4bit --dtype float16 --gpu-memory-utilization 0.8 --max-model-len 13500 --quantization bitsandbytes --load-format bitsandbytes --served-model-name meta-llama/Llama-3.1-8B --trust-remote-code --chat-template llama_3.1_template.jinja

# vllm serve unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit --dtype float16 --gpu-memory-utilization 0.8 --max-model-len 13500 --quantization bitsandbytes --load-format bitsandbytes --served-model-name meta-llama/Llama-3.1-8B-Instruct --trust-remote-code

# vllm serve unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit \
#  --dtype float16 --gpu-memory-utilization 0.8 --max-model-len 13500 --quantization bitsandbytes \
#  --load-format bitsandbytes --served-model-name meta-llama/Llama-3.1-8B-Instruct --trust-remote-code \
#  --enable-lora \
#  --lora-modules mylora120steps=./lora_adapter_llama8b_120


# vllm serve unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit \
#  --dtype float16 --gpu-memory-utilization 0.8 --max-model-len 13500 --quantization bitsandbytes \
#  --load-format bitsandbytes --served-model-name meta-llama/Llama-3.1-8B-Instruct --trust-remote-code \
#  --enable-lora \
#  --lora-modules mylora3epochs=./lora_adapter_llama8b_30epochs

# vllm serve meta-llama/Llama-2-7b-hf \
   

In [5]:

lm = dspy.LM('openai/meta-llama/Llama-3.1-8B-Instruct',api_base="http://localhost:8000/v1",  # ensure this points to your port
             api_key="local", model_type='chat')
dspy.configure(lm=lm)

In [6]:
lm("Hello 12")

["It seems like you're trying to initiate a conversation. I'd be happy to chat with you. How's your day going so far?"]

In [16]:
from typing import Literal

class Categorize(dspy.Signature):
    """Solve exam problem."""

    question: str = dspy.InputField()
    options: list[dict[str,str]] = dspy.InputField()
    similar_examples : list[str] = dspy.InputField()
    correct_marker: str = dspy.OutputField()

classify = dspy.Predict(Categorize, False,cache=False)
# classify = dspy.ChainOfThought(Categorize, False,cache=False)

# classify.load("BootstrapFewShotWithRandomSearch_program_15_candidates_llama.json")

# Here is how we call this module
# classification = classify(question = all_examples[-1].question, options = all_examples[-1].options, cache=False)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v3",model_kwargs = {"trust_remote_code": True})

vector_store = Chroma(
    collection_name="ukrainian_language",
    persist_directory="./chroma_db",
    embedding_function=embeddings,
)



In [8]:
sample = test_set[1]
print(sample.question)

retrieved_docs = vector_store.similarity_search(sample.question)
docs_list = [doc.page_content for doc in retrieved_docs]

docs_list

Позначте словосполучення, у якому порушено граматичну норму на позначення часу:


["Question: НЕПРАВИЛЬНО побудовано словосполучення на позначення часу\nOptions:[{'marker': 'А', 'text': 'десять (хвилин) на десяту'}, {'marker': 'Б', 'text': 'чверть по одинадцятій'}, {'marker': 'В', 'text': 'третя година десять хвилин'}, {'marker': 'Г', 'text': 'за п’ять (хвилин) шоста'}, {'marker': 'Д', 'text': 'без п’ятнадцяти десять'}]\nAnswer Д",
 "Question: Неправильно побудовано словосполучення на позначення часу\nOptions:[{'marker': 'А', 'text': 'десять (хвилин) на десяту'}, {'marker': 'Б', 'text': 'чверть на одинадцяту'}, {'marker': 'В', 'text': 'без п’ятнадцяти десять'}, {'marker': 'Г', 'text': 'пів на дванадцяту'}, {'marker': 'Д', 'text': 'за п’ять (хвилин) шоста'}]\nAnswer В",
 'Question: Граматично правильно утворено словосполучення\nOptions:[{\'marker\': \'А\', \'text\': \'розпочнеться в четвертій годині\'}, {\'marker\': \'Б\', \'text\': "зустрінемося за чверть дев\'ята"}, {\'marker\': \'В\', \'text\': \'тренувалися до пів сьомої години\'}, {\'marker\': \'Г\', \'text\': \

In [9]:
sample['options']

[{'marker': 'А', 'text': 'рівно о першій;'},
 {'marker': 'Б', 'text': 'десять хвилин по шостій;'},
 {'marker': 'В', 'text': 'пів на десяту;'},
 {'marker': 'Г', 'text': 'сім годин двадцять хвилин;'},
 {'marker': 'Д', 'text': 'за двадцять п’ята.'}]

In [17]:
from tqdm import tqdm

classifications = []
correct_answers = []
for sample in tqdm(test_set):

    if sample.subject == "ukrainian-language-and-literature":
        retrieved_docs = vector_store.similarity_search(sample.question)
        docs_list = [doc.page_content for doc in retrieved_docs]
    elif sample.subject == "history-of-ukraine":
        retrieved_docs = vector_store.similarity_search(sample.question)
        docs_list = [doc.page_content for doc in retrieved_docs]
    else:
        docs_list = []

    classification = classify(question = sample.question, options = sample.options, similar_examples=docs_list)
    classifications.append(classification.correct_marker)
    correct_answers.append(sample.correct_answer)
    break

  0%|          | 0/612 [00:00<?, ?it/s]


In [27]:
allowed_answers = ['А', 'Б', 'В', 'Г', 'Д', '1', '2', '3', '4', '5', '6', '7', '8', '9']

def post_process_sample(prediction:str) -> str:
    found_answer = 'А'  # Default value

    for allowed_answer in allowed_answers:
        if allowed_answer in prediction:
            found_answer = allowed_answer
            # print(f"answer {found_answer} for {classification}")
            break
    return found_answer

post_processed = []

for classification in classifications:
    found_answer = post_process_sample(classification)
    post_processed.append(found_answer)

In [28]:
len(post_processed)

612

In [29]:
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

def evaluate_model(predictions, ground_truth):
    """
    Simple evaluation of classification model.
    
    Parameters:
    predictions (list): Model's predicted answers
    ground_truth (list): Correct answers
    """
    # Calculate accuracy
    accuracy = accuracy_score(ground_truth, predictions)
    
    # Get unique options
    unique_options = sorted(list(set(predictions + ground_truth)))
    
    # Create confusion matrix
    cm = confusion_matrix(ground_truth, predictions, labels=unique_options)
    
    # Print results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"\nConfusion Matrix:")
    print("Options:", unique_options)
    print(cm)

## Evaluations

In [30]:
#RAG ON train data with COT
evaluate_model(classifications,correct_answers)

Accuracy: 0.2598

Confusion Matrix:
Options: ['', '1', '4', '[[ marker "В" ]]', '[[ marker: "Г" ]]', '[[ marker: "Д", text: "бігти — біжуть" ]]', '[[marker: "Б"]]', '[[marker: "Г"]]', 'А', 'А, Б, В, Г, Д', 'Б', 'В', 'Варіант "Г" містить прийменник "за", який вказує місце дії.', 'Варіант А', 'Вибираємо відповідь, яка відповідає бажанню героя «списа приломити кінець поля Половецького». У подібних прикладах ми бачимо, що відповіддю на подібний запит був «Всеволод».', 'Вибираємо прикметник, який вказує на якість або стан чогось. У даному тексті такі прикметники як "зела", "садів", "свята", "таїна". Але тільки одне з них є правильним відповідником прикметника у даному тексті. Правильний відповідник прикметника у даному тексті - "свята".', 'Вибір правильного варіанту відповідає на питання про тип мовлення тексту. Правильний варіант відповідає на питання про тип мовлення тексту, який є розповіддю.', 'Вибір правильного варіанту залежить від змісту кожного варіанту. Варіант А відповідає змісту 

In [31]:
#RAG ON train data with COT
evaluate_model(post_processed,correct_answers)

Accuracy: 0.2794

Confusion Matrix:
Options: ['1', '4', 'А', 'Б', 'В', 'Г', 'Д']
[[ 0  0  1  0  0  0  0]
 [ 0  0  1  0  0  0  0]
 [ 0  0 64  4 21 21  2]
 [ 0  0 74 23 39 21  8]
 [ 0  0 57 15 38 24  9]
 [ 0  0 55 10 25 34  7]
 [ 0  0 30  3  6  8 12]]


In [16]:
#RAG ON train data
evaluate_model(classifications,correct_answers)

Accuracy: 0.2908

Confusion Matrix:
Options: ['"Б"', '(1)', '1', '4', 'А', 'Б', 'В', 'Василь Стус', 'Г', 'Д']
[[ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0]
 [ 0  1  0  0  0  0  0  0  0  0]
 [ 1  0  0  0 15 10 55  0 29  2]
 [ 0  0  0  0  5 22 96  0 37  5]
 [ 0  0  0  0  9  8 95  1 28  2]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 12 17 60  1 38  3]
 [ 0  0  0  0  3  3 31  0 14  8]]


In [17]:
#RAG ON train data
evaluate_model(post_processed,correct_answers)

Accuracy: 0.2925

Confusion Matrix:
Options: ['1', '4', 'А', 'Б', 'В', 'Г', 'Д']
[[ 0  0  0  0  0  1  0]
 [ 1  0  0  0  0  0  0]
 [ 0  0 15 11 55 29  2]
 [ 0  0  5 22 96 37  5]
 [ 0  0  9  8 96 28  2]
 [ 0  0 12 17 61 38  3]
 [ 0  0  3  3 31 14  8]]


In [18]:
#untrained rag for ukr lan and history
evaluate_model(classifications,correct_answers)

Accuracy: 0.2418

Confusion Matrix:
Options: ['"І чужому научайтесь, й свого не цурайтесь" є в творі "Ісаія. Глава 35"', '"А"', '"Б"', '"В"', '"В. Тропінін, Т. Шевченко, І.Сошенко, В.Боровиковський"', '"Вона зрозуміла душевний *стан* гостя. У хлопця плечі широкі, *стан* високий."', '"Г"', '"Д"', '"Мина Мазайло («Мина Мазайло» Миколи Куліша)"', '"Сумління людське, виявляється, схоже на вулик, його теж можна розтривожити (О. Гончар)."', '"е"', '1', '17 вересня 1939 р.', '4', 'А', 'А, А, А, А, А', 'Б', 'В', 'В. Філатов і М. Стражеско, генетики М. Холодний та О. Сапєгін, фізики Л. Ландау і П. Капиця, історики М. Грушевський і Д. Баталій, економіст М. Волобуєв', 'Василеві Симоненку', 'Василь Стус', 'Велике розселення слов’ян тривало протягом V–VII ст.', 'Вельмишановна Валеріє Сергіївно!', 'Вибач мені *за/те*, що була неввічлива.', 'Вибух на четвертому енергоблоці Чорнобильської АЕС стався в квітні 1986 р.', 'Визволення УРСР від нацистських загарбників у її довоєнних кордонах здійснено в ход

In [19]:
#untrained rag for ukr lanand history
evaluate_model(post_processed,correct_answers)

Accuracy: 0.2549

Confusion Matrix:
Options: ['1', '4', 'А', 'Б', 'В', 'Г', 'Д']
[[ 1  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0]
 [ 0  0 27 12 47 17  9]
 [ 0  0 37 24 70 15 19]
 [ 0  0 30 12 75 19  7]
 [ 1  0 33  7 62 19  9]
 [ 0  0 13  3 28  5 10]]


In [15]:
#untrained rag for ukr lan
evaluate_model(classifications,correct_answers)

Accuracy: 0.2500

Confusion Matrix:
Options: ['"І чужому научайтесь, й свого не цурайтесь" є в творі "Ісаія. Глава 35"', '"А"', '"В"', '"Вона зрозуміла душевний *стан* гостя. У хлопця плечі широкі, *стан* високий."', '"Г"', '"Д"', '"Мина Мазайло («Мина Мазайло» Миколи Куліша)"', '"е"', '"соціально-психологічний роман"', '1', '4', 'None', 'А', 'А, А, А, А, А', 'Б', 'В', 'В. Симоненка, І. Світличного.', 'Василь Стус', 'Вельмишановний пан лікар!', 'Вибач мені *за/те*, що була неввічлива.', 'Визволення УРСР від нацистських загарбників у її довоєнних кордонах здійснено в ході', 'Виправлення: "Виділене слово вжите в прямому значенні в реченні"', 'Виправте речення, замінивши тире між підметом і присудком на відповідний розділовий знак.', 'Виражений дієприкметниковий зворот у реченні: "За Дніпром небо ясно палало всіма кольорами веселки."', 'Всеволод', 'Віршовий розмір «Енеїди» Івана Котляревського — дактиль.', 'Г', 'Д', 'Дуже сильно діяти', 'Початковий текст містить кілька речень, які потрібн

In [16]:
#untrained rag for ukr lan
evaluate_model(post_processed,correct_answers)

Accuracy: 0.2565

Confusion Matrix:
Options: ['1', '4', 'А', 'Б', 'В', 'Г', 'Д']
[[ 1  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0]
 [ 0  0 27 12 50 15  8]
 [ 0  0 34 30 64 16 21]
 [ 0  0 31 14 73 16  9]
 [ 0  0 33 10 56 19 13]
 [ 0  0 14  3 29  6  7]]


In [39]:
evaluate_model(classifications,correct_answers)

Accuracy: 0.2565

Confusion Matrix:
Options: ['"А"', '"Б"', '"В"', '(1)Уміння радіти чужим успіхам мірило шляхетності. (2)Якщо людину підтримати похвалити підкреслити щось хороше в ній вона намагається дорівнювати уявленню яке про неї склалося. (3)Ось чому так важливо з дитинства культивувати в людині почуття власної гідності відчуття певної „собівартості” аби згодом не склався комплекс меншовартості. (4)Недарма кажуть Якщо казати людині постійно що вона свиня то згодом вона зарохкає. (5)Не ввічливо кидати компліменти малознайомим або незнайомим людям. (6)Краще зачекати познайомившись ближче чи навіть заприятелювавши з ними. (7)Але пам’ятаймо що слова похвали потрібні кожному.', '1', '4', '[]', 'А', 'А, Б, В, Г, Д', 'А-1, Б-2, В-3, Г-4, Д-5', 'АВГД', 'Б', 'В', 'Василеві Стусу', 'Г', 'Д']
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  

In [61]:
evaluate_model(post_processed,correct_answers)

Accuracy: 0.2712

Confusion Matrix:
Options: ['1', '4', 'А', 'Б', 'В', 'Г', 'Д']
[[ 0  0  1  0  0  0  0]
 [ 0  0  1  0  0  0  0]
 [ 0  0 16 17 67 11  1]
 [ 0  0 17 39 90 14  5]
 [ 0  0 14 24 88 15  2]
 [ 0  0 10 23 77 20  1]
 [ 0  0  6  6 39  5  3]]
